# IPython Blockfolio

This is a simple "Blockfolio" using Python and the [CoinMarketCap API](https://coinmarketcap.com/api/).

It will show the net worth in USD of your Cryptocurrency investments as well as the relative share
of coins you own in comparison to the total coin supply of the corresponding digital asset.

First, the current exchanges rates are loaded, then combined with your investments, and finally displayed using Plotly.

Imports
--------

In [1]:
import pandas as pd

import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)

## Loading Data from CoinMarkteCap

In [2]:
limit = 200  # only load the top 200 currencies, if you have invested in smaller ones increase this limit
coin_market_cap_api = 'https://api.coinmarketcap.com/v1/ticker/?limit={}'.format(limit)
coin_market_cap_api

'https://api.coinmarketcap.com/v1/ticker/?limit=200'

In [3]:
# load the data using pandas
market_data = pd.read_json(coin_market_cap_api)
market_data.head(10)

,24h_volume_usd,available_supply,id,last_updated,market_cap_usd,max_supply,name,percent_change_1h,percent_change_24h,percent_change_7d,price_btc,price_usd,rank,symbol,total_supply
0,1.128680e+10,16769250,bitcoin,1514539460,247505745375,2.100000e+07,Bitcoin,-0.42,1.24,7.67,1.000000,14759.500000,1,BTC,16769250
1,2.413760e+09,96638451,ethereum,1514539448,72213952348,NaN,Ethereum,-0.47,3.51,10.54,0.051276,747.259000,2,ETH,96638451
2,3.261180e+09,38739144847,ripple,1514539441,62191048354,1.000000e+11,Ripple,2.29,19.25,49.97,0.000110,1.605380,3,XRP,99993093880
3,3.558350e+09,16881900,bitcoin-cash,1514539452,42310430694,2.100000e+07,Bitcoin Cash,-0.70,-5.32,0.37,0.171975,2506.260000,4,BCH,16881900
4,3.035370e+09,54521783,litecoin,1514539441,13384988721,8.400000e+07,Litecoin,-1.37,-3.64,-3.03,0.016846,245.498000,5,LTC,54521783
5,1.024840e+08,25927070538,cardano,1514539454,11333344782,4.500000e+10,Cardano,1.26,13.29,11.73,0.000030,0.437124,6,ADA,31112483745
6,1.963430e+08,2779530283,iota,1514539451,10325232323,2.779530e+09,IOTA,0.79,6.74,-0.00,0.000255,3.714740,7,MIOTA,2779530283
7,1.618950e+08,7781828,dash,1514539442,8686699078,1.890000e+07,Dash,-0.74,1.45,0.44,0.076597,1116.280000,8,DASH,7781828
8,4.714140e+07,8999999999,nem,1514539444,8240408999,NaN,NEM,-1.86,4.42,12.13,0.000063,0.915601,9,XEM,8999999999
9,1.372670e+08,15529227,monero,1514539443,5754059877,NaN,Monero,-0.75,-0.93,13.46,0.025425,370.531000,10,XMR,15529227


## Your Blockfolio

Below you can fill in the types and amount of coins you have into the Python dictionary
with the format `'SYMBOL': value`.

In [13]:
# enter your coins below:

block_folio = {
    'BTC': 0.31337,
    'BCH': 0.01337,
    'ETH': 3,
    'LTC': 1.5,
    'MIOTA': 777,
    'XRP': 13,
    'DASH': 0.1, 
    'XMR': 1, 
    'LSK': 12, 
    'OMG': 1.1, 
    'XRB': 42, 
    'PIVX': 123, 
    'ARK': 22, 
    'NEO': 7, 
    'GAS': 0.01,
    'STEEM': 499,
}

# display your blockfolio
block_folio = pd.DataFrame(list(block_folio.items()), columns=['symbol', 'amount'])
block_folio.head(len(block_folio))

,symbol,amount
0,GAS,0.01000
1,STEEM,499.00000
2,OMG,1.10000
3,BTC,0.31337
4,DASH,0.10000
5,ETH,3.00000
6,MIOTA,777.00000
7,LSK,12.00000
8,PIVX,123.00000
9,LTC,1.50000


### Combining the Blockfolio and the API Data

In [14]:
# merge the API and blockfolio data and sort by investment value
merged_data = block_folio.merge(market_data, how='left')
merged_data['value_usd'] = merged_data.amount * merged_data.price_usd
merged_data['coinshare'] = merged_data.amount / merged_data.available_supply
merged_data = merged_data.sort_values('value_usd', ascending=False)
merged_data.head()

,symbol,amount,24h_volume_usd,available_supply,id,last_updated,market_cap_usd,max_supply,name,percent_change_1h,percent_change_24h,percent_change_7d,price_btc,price_usd,rank,total_supply,value_usd,coinshare
3,BTC,0.31337,1.128680e+10,16769250,bitcoin,1514539460,247505745375,2.100000e+07,Bitcoin,-0.42,1.24,7.67,1.000000,14759.50000,1,16769250,4625.184515,1.868718e-08
6,MIOTA,777.00000,1.963430e+08,2779530283,iota,1514539451,10325232323,2.779530e+09,IOTA,0.79,6.74,-0.00,0.000255,3.71474,7,2779530283,2886.352980,2.795436e-07
5,ETH,3.00000,2.413760e+09,96638451,ethereum,1514539448,72213952348,NaN,Ethereum,-0.47,3.51,10.54,0.051276,747.25900,2,96638451,2241.777000,3.104354e-08
1,STEEM,499.00000,4.569410e+06,246336993,steem,1514539447,704565677,NaN,Steem,-0.79,-0.44,5.33,0.000196,2.86017,41,263311087,1427.224830,2.025680e-06
8,PIVX,123.00000,7.298730e+06,55233298,pivx,1514539446,605268575,NaN,PIVX,-0.67,-5.24,38.98,0.000752,10.95840,45,55233298,1347.883200,2.226918e-06


### Your Blockfolio in numbers and graphs!


In [15]:
networth = 'Your blockfolio is currently worth {:.2f} USD!'.format(merged_data.value_usd.sum())
print(networth)

Your blockfolio is currently worth 14974.87 USD!


#### Net Worth

Let's plot the worth of each asset in descending order:

In [18]:
marker = dict(color='rgb(158,202,225)',
              line=dict(color='rgb(8,48,107)', width=1.5))

layout = go.Layout(title='Blockfolio Networth in USD',
                   xaxis=dict(title='Cryptocurrency',),
                   yaxis=dict(title='USD'))
              
value_chart=go.Bar(x=merged_data.symbol, y=merged_data.value_usd, marker=marker)

fig = go.Figure(data=[value_chart], layout=layout)
py.iplot(fig)

#### Fraction of total Supply
Finally, that looks at the relative sizes of your investment. 
This gives you an idea or visualizes in which cryptocurrency you believe in the most. 
The graph below shows you fractional share of the the current crypto's total supply.

In [20]:
marker = dict(color='rgb(258,202,125)', line=dict(color='rgb(8,48,107)', width=1.5,))

layout = go.Layout(title='Relative Blockfolio Size',
                   xaxis=dict(title='Cryptocurrency',),
                   yaxis=dict(title='Fraction of total Supply'))

share_chart=go.Bar(x=merged_data.symbol, y=merged_data.coinshare, marker=marker)

fig = go.Figure(data=[share_chart], layout=layout)
py.iplot(fig)